In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from random import randint

In [2]:
#colunas que escolhi para dados escolares
col_escol = ['CODMEC','ANO','TIPOESC','DEP','Qtde_Docentes','NUM_FUNC','MERENDA','CLFUND',
            'ALFUND','TOTALEJA','TOT_EE','TOTAL_ALUNOS','CL_CRECHE','AL_CRECHE','CL_PRE','AL_PRE','CLEM','ALEM']
col_rendi = ['CODMEC','EFABAND9ANOS8S','EFABAND9ANOS9S','EFABAND7S','EFABAND8S','TOTALEF_ABAND','TOTALEF_APROV',
             'TOTALEF_REPROV','TOTAL_EM_ABAND','TOTAL_EM_REPROV','CONCLUINTES_EM','EPABAND','EPREPROV']
dados_escolares = pd.read_csv('DadosEscolares1996-2015.csv', encoding = 'latin1', usecols = col_escol)
rendimento = pd.read_csv('RendimentoEscolar2000-2015.csv', encoding = 'latin1', usecols = col_rendi)
esc_rmsp = pd.read_csv('ESC2013_RMSP_CEM.csv', encoding = 'latin1')
nota = pd.read_csv('ENEM2015.csv')
#como eu abri todas as colunas do esc_rmsp, agora vou retirar as que não desejo usar
col_rmsp = ['ID', 'LONGITUDE', 'LATITUDE', 'CODESCTX','NOMEESC', 'NOMEMUN','CODMUN', 'NOMDIST', 'CODDIST', 'COD_SC',
           'BAIRRO','CEP','END_ESC','NUM_ESC','DDD','TELEFONE','ENS_REG','REG_IN_CR','REG_IN_PE','REG_FU_8A',
           'REG_FU_9A','REG_MED','REG_MED_I','REG_MED_N','REG_MED_P','ENS_ESP','ESP_IN_CR','ESP_IN_PE',
           'ESP_FU_8A','ESP_FU_9A','ESP_MED','ESP_MED_I','ESP_MED_N','ESP_MED_P','ESP_EJA_F','ESP_EJA_M','EJA',
           'EJA_FUND','EJA_MED','EJA_PROJ']
esc_rmsp = esc_rmsp.drop(col_rmsp, axis = 1)

C:\Users\giova\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(104292, 18)
Rendimento(14944, 13)
ESC_RMSP(10937, 96)
Nota(1243, 3)


In [4]:
#def t(x):
#    if x==0:
#        return None
#    else:
#        return x
#for c in dados_escolares.columns:
#    dados_escolares[c] = dados_escolares[c].apply(t)

Agrupei os dados nos três dataframes por escola fazendo a média das linhas.

In [5]:
#Agrupo por escolas
dados_escolares = dados_escolares.groupby('CODMEC').mean()
rendimento = rendimento.groupby('CODMEC').mean()
esc_rmsp = esc_rmsp.groupby('CODESC').mean()

In [6]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(9642, 14)
Rendimento(4892, 12)
ESC_RMSP(10937, 93)
Nota(1243, 3)


Apaguei as linhas que tinham a coluna Classificação igual a 0 (apenas 3 linhas) e apaguei todos os dados de escolas que não estão no target.

In [7]:
#Apagando a classe desbalanceada
nota = nota[nota.Classificacao != 0]
#Em todos Dataframes temos mais escolas do que o nosso target
dados_escolares = dados_escolares.reset_index()[dados_escolares.reset_index().CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
rendimento = rendimento.reset_index()[rendimento.reset_index().CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
esc_rmsp = esc_rmsp.reset_index()[esc_rmsp.reset_index().CODESC.isin(nota['CODIGO DA ENTIDADE'])]

In [8]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(692, 15)
Rendimento(504, 13)
ESC_RMSP(1241, 94)
Nota(1241, 3)


Apaguei as colunas que tinham mais do que 15% de valores NaN e depois apaguei as linhas que tinham algum valor NaN.

In [9]:
#apagando as linha com valor NaN
dados_escolares.dropna(inplace = True, how = 'any')
esc_rmsp.drop(esc_rmsp.columns[esc_rmsp.isnull().mean() > 0.15], axis=1, inplace = True)
esc_rmsp.dropna(how='any', axis = 0, inplace = True)

In [10]:
#verificando o tamanho
print ( 'Dados escolares' + str(dados_escolares.shape))
print ( 'Rendimento' + str(rendimento.shape))
print ( 'ESC_RMSP' + str(esc_rmsp.shape))
print( 'Nota' + str(nota.shape))

Dados escolares(690, 15)
Rendimento(504, 13)
ESC_RMSP(931, 21)
Nota(1241, 3)


Utilizando apenas o dataframe dados_escolares.

In [11]:
#Alterando o nome da coluna para o merge
nota.rename(columns = {'CODIGO DA ENTIDADE':'CODMEC'}, inplace = True)
#criando o merge com base na coluna CODMEC usando o dataframe target e os dados escolares
new_df = pd.merge(dados_escolares, nota, on = 'CODMEC')
new_df.drop(['CODMEC', 'ANO','NUMERO DE PARTICIPANTES'], axis = 1, inplace = True)

In [12]:
#criando o data e o target
data = new_df.drop('Classificacao', axis = 1)
target = new_df.Classificacao

Realizo um teste com os dados normalizados, utilizando KFolds e linear regression.

In [13]:
lm = LinearRegression()
scaler = StandardScaler()
data1 = scaler.fit_transform(data)
#PCA não melhora o score
kf = KFold(5, shuffle = True)
r = []
for train_i, test_i in kf.split(data):
    x_train, x_test = data1[train_i], data1[test_i]
    y_train, y_test = target[train_i], target[test_i]
    lm.fit(x_train, y_train)
    r.append(lm.score(x_test, y_test))
r.sort(reverse = True)
print('Kfolds results: ', r)
print('Score mean: ', sum(r)/5)

Kfolds results:  [0.4720345159024039, 0.4682137357581474, 0.4570959709764666, 0.3389382795206265, 0.31334855487962526]
Score mean:  0.40992621140745394


In [114]:
def modelo_svm_kf(data, target, gamma, C, k):
    kf = KFold(n_splits = k)
    scores = []
    for g in gamma:
            for c in C:
                mean_kf = []
                for train_i, test_i in kf.split(data):
                    #separação dos dados em treino e teste
                    data_train, data_test = data.iloc[train_i], data.iloc[test_i]
                    target_train, target_test = target[train_i], target[test_i]
                    
                    #normalização dos dados
                    scaler = StandardScaler()
                    scaler.fit(data_train)
                    data_train = scaler.transform(data_train)
                    data_test = scaler.transform(data_test)
                    
                    #pca nos dados
                    pca = PCA(0.95)
                    pca.fit(data_train)
                    data_train = pca.transform(data_train)
                    data_test = pca.transform(data_test)
                    
                    #rodando o svm
                    svc = SVC(kernel = 'rbf', C=c, gamma = g)
                    svc.fit(data_train,target_train)
                    mean_kf.append(svc.score(data_test,target_test))
                                   
                #salvando a média do KFold para g e c dados
                scores.append(((sum(mean_kf)/len(mean_kf)), g, c))
                print('Iteração com gamma %f e c %f concluída' %(g, c))
    return scores

Realizo um teste usando:
- Kfolds
-  Normalização
- PCA
- SVM
- Tuning dos parametros

In [115]:
res = modelo_svm_kf(data, target, gamma = [0.01, 0.1, 1, 10, 100], C = [0.01, 0.1, 1, 10, 100], k = 5)
res.sort()
print('Maiores score e os parâmetros:')
res[-5:]

Iteração com gamma 0.010000 e c 0.010000 concluída
Iteração com gamma 0.010000 e c 0.100000 concluída
Iteração com gamma 0.010000 e c 1.000000 concluída
Iteração com gamma 0.010000 e c 10.000000 concluída
Iteração com gamma 0.010000 e c 100.000000 concluída
Iteração com gamma 0.100000 e c 0.010000 concluída
Iteração com gamma 0.100000 e c 0.100000 concluída
Iteração com gamma 0.100000 e c 1.000000 concluída
Iteração com gamma 0.100000 e c 10.000000 concluída
Iteração com gamma 0.100000 e c 100.000000 concluída
Iteração com gamma 1.000000 e c 0.010000 concluída
Iteração com gamma 1.000000 e c 0.100000 concluída
Iteração com gamma 1.000000 e c 1.000000 concluída
Iteração com gamma 1.000000 e c 10.000000 concluída
Iteração com gamma 1.000000 e c 100.000000 concluída
Iteração com gamma 10.000000 e c 0.010000 concluída
Iteração com gamma 10.000000 e c 0.100000 concluída
Iteração com gamma 10.000000 e c 1.000000 concluída
Iteração com gamma 10.000000 e c 10.000000 concluída
Iteração com gamm

[(0.7130434782608696, 100, 0.01),
 (0.7130434782608696, 100, 0.1),
 (0.7130434782608696, 100, 1),
 (0.7159420289855072, 0.01, 100),
 (0.7246376811594203, 0.01, 1)]

Repito o mesmo erro que tive no outro dia, utilizando tanto o KFolds quanto o train_test_split.

In [116]:
params = {'kernel': ['rbf'], 'C':[0.1], 'gamma':[100]}
svr = SVR()
clf = GridSearchCV(estimator = svr, param_grid = params, cv = 5, return_train_score=True)
clf.fit(data, target)
print( 'Treino: ', clf.cv_results_['mean_train_score'])
print( 'Teste: ', clf.cv_results_['mean_test_score'])

Treino:  [0.14075827]
Teste:  [-0.22302813]


In [117]:
svr = SVR(kernel = 'rbf', C=0.1, gamma = 100)
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size = 0.2)
svr.fit(x_train, y_train)
print( 'Treino: ', svr.score(x_train, y_train))
print( 'Teste: ', svr.score(x_test, y_test))

Treino:  0.14713908838907208
Teste:  -0.0019170975005526891


In [118]:
target.value_counts()

2    492
3     84
1     74
4     40
Name: Classificacao, dtype: int64

Faço o balanceamento das classes, agora tenho 40 escolas com cada classe, cada classe agora tem 40 escolas.

In [19]:
#fazendo o balanceamento
balanced = new_df.groupby('Classificacao').apply(lambda x: x.sample(40))
balanced = balanced.sample(frac = 1)
balanced = balanced.droplevel(level = 0)
balanced = balanced.reset_index()
balanced.drop(['index'], axis = 1, inplace = True)
data_balanced = balanced.drop(['Classificacao'], axis =1)
target_balanced = balanced.Classificacao

Realizo um teste usando:
- Classes balanceadas
- Kfolds
-  Normalização
- PCA
- SVM
- Tuning dos parametros

In [20]:
r = modelo_svm_kf(data_balanced, target_balanced, gamma = [0.01, 0.1, 1, 10,100],C = [0.01, 0.1, 1, 10, 100],k = 5)
r.sort()
r[-5:]

Iteração com gamma 0.010000 e c 0.010000 concluída
Iteração com gamma 0.010000 e c 0.100000 concluída
Iteração com gamma 0.010000 e c 1.000000 concluída
Iteração com gamma 0.010000 e c 10.000000 concluída
Iteração com gamma 0.010000 e c 100.000000 concluída
Iteração com gamma 0.100000 e c 0.010000 concluída
Iteração com gamma 0.100000 e c 0.100000 concluída
Iteração com gamma 0.100000 e c 1.000000 concluída
Iteração com gamma 0.100000 e c 10.000000 concluída
Iteração com gamma 0.100000 e c 100.000000 concluída
Iteração com gamma 1.000000 e c 0.010000 concluída
Iteração com gamma 1.000000 e c 0.100000 concluída
Iteração com gamma 1.000000 e c 1.000000 concluída
Iteração com gamma 1.000000 e c 10.000000 concluída
Iteração com gamma 1.000000 e c 100.000000 concluída
Iteração com gamma 10.000000 e c 0.010000 concluída
Iteração com gamma 10.000000 e c 0.100000 concluída
Iteração com gamma 10.000000 e c 1.000000 concluída
Iteração com gamma 10.000000 e c 10.000000 concluída
Iteração com gamm

[(0.525, 0.1, 100),
 (0.58125, 0.1, 10),
 (0.5875, 0.1, 1),
 (0.59375, 0.01, 10),
 (0.60625, 0.01, 100)]

Percebo que o erro é o valor de C.

In [21]:
scaler = StandardScaler()
data_balanced1 = scaler.fit_transform(data_balanced)
params = {'kernel': ['rbf'], 'C':[0.01, 0.1, 1, 10,100], 'gamma':[0.01, 0.1, 1, 10, 100]}
svr = SVR()
clf = GridSearchCV(estimator = svr, param_grid = params, cv = 5, return_train_score=True)
clf.fit(data_balanced1, target_balanced)
print( 'Treino: ', clf.cv_results_['mean_train_score'])
print( 'Teste: ', clf.cv_results_['mean_test_score'])

Treino:  [-0.0580772  -0.01018249 -0.06643596 -0.08936642 -0.09031282  0.26968844
  0.43519755  0.24234908  0.09276661  0.08441945  0.57497508  0.75545311
  0.90725108  0.90369238  0.89566295  0.69253092  0.8755207   0.99133176
  0.99204367  0.9919884   0.81158077  0.94852189  0.99234063  0.99204367
  0.9919884 ]
Teste:  [-0.16309458 -0.1114864  -0.18147225 -0.21320515 -0.21425981  0.20797829
  0.35197897  0.0782364  -0.14526022 -0.15494523  0.52886327  0.59083284
  0.39354005  0.0048188  -0.02429771  0.61213792  0.58434184  0.40467673
  0.01259529 -0.01900778  0.60229486  0.34265341  0.40794645  0.01259529
 -0.01900778]


In [23]:
scaler = StandardScaler()
params = {'kernel': ['rbf'], 'C':[0.01, 0.1, 1, 10,100], 'gamma':[0.01, 0.1, 1, 10, 100]}
svr = SVR()
clf = GridSearchCV(estimator = svr, param_grid = params, cv = 5, return_train_score=True, refit = True)
clf.fit(data1, target)
print( 'Treino: ', clf.cv_results_['mean_train_score'])
print( 'Teste: ', clf.cv_results_['mean_test_score'])

Treino:  [0.04644468 0.06310087 0.01374831 0.0030881  0.00290729 0.23926396
 0.26830185 0.18483379 0.14251386 0.14076896 0.40153691 0.55298718
 0.79866586 0.88543061 0.88414376 0.48684638 0.69091509 0.95782899
 0.97803488 0.97712555 0.55408799 0.81116576 0.97864212 0.97803488
 0.97712555]
Teste:  [-0.2018141  -0.19576325 -0.21402055 -0.21018039 -0.21031691 -0.0402193
 -0.0683221  -0.20963459 -0.22233695 -0.22309383  0.17263795  0.08401124
 -0.1921403  -0.351319   -0.3398982   0.17189969 -0.17032501 -0.29433483
 -0.35775385 -0.36275587 -0.00449163 -0.83165966 -0.48803898 -0.35775385
 -0.36275587]


Utilizando apenas o dataframe esc_rmsp.

In [83]:
esc_rmsp.rename(columns={'CODESC':'CODMEC'}, inplace = True)
new_df = pd.merge(esc_rmsp, nota, on = 'CODMEC')
new_df.drop(['NUMERO DE PARTICIPANTES', 'CODMEC'], axis = 1,inplace = True)
data = new_df.drop(['Classificacao'], axis =1)
target = new_df.Classificacao
data.COD_DEP = data.COD_DEP.astype('float64')
data.LOCALIZA = data.LOCALIZA.astype('float64')
data.LAB_INF = data.LAB_INF.astype('float64')
data.QD_ESP_C = data.QD_ESP_C.astype('float64')
data.QD_ESP_D = data.QD_ESP_D.astype('float64')
data.BIBLIO = data.BIBLIO.astype('float64')

Realizo um teste usando Kfolds, normalização e linear regression.

In [85]:
lm = LinearRegression()
scaler = StandardScaler()
data1 = scaler.fit_transform(data)
#PCA não melhora o score
kf = KFold(5, shuffle=True)
r = []
for train_i, test_i in kf.split(data):
    x_train, x_test = data1[train_i], data1[test_i]
    y_train, y_test = target[train_i], target[test_i]
    lm.fit(x_train, y_train)
    r.append(lm.score(x_test, y_test))
r.sort(reverse = True)
print('Kfolds results: ', r)
print('Score mean: ', sum(r)/5)

Kfolds results:  [0.5519871022599485, 0.5418561407000198, 0.5358635778891673, 0.46773194882564934, 0.40969403453438324]
Score mean:  0.5014265608418336


Realizo um teste usando:
- Kfolds
-  Normalização
- PCA
- SVM
- Tuning dos parametros

In [95]:
res = modelo_svm_kf(data, target, gamma = [0.01, 0.1, 1, 10, 100], C = [0.01, 0.1, 1, 10, 100], k = 5)
res.sort()
print('Maiores score e os parâmetros:')
res[-5:]

Iteração com gamma 0.010000 e c 0.010000 concluída
Iteração com gamma 0.010000 e c 0.100000 concluída
Iteração com gamma 0.010000 e c 1.000000 concluída
Iteração com gamma 0.010000 e c 10.000000 concluída
Iteração com gamma 0.010000 e c 100.000000 concluída
Iteração com gamma 0.100000 e c 0.010000 concluída
Iteração com gamma 0.100000 e c 0.100000 concluída
Iteração com gamma 0.100000 e c 1.000000 concluída
Iteração com gamma 0.100000 e c 10.000000 concluída
Iteração com gamma 0.100000 e c 100.000000 concluída
Iteração com gamma 1.000000 e c 0.010000 concluída
Iteração com gamma 1.000000 e c 0.100000 concluída
Iteração com gamma 1.000000 e c 1.000000 concluída
Iteração com gamma 1.000000 e c 10.000000 concluída
Iteração com gamma 1.000000 e c 100.000000 concluída
Iteração com gamma 10.000000 e c 0.010000 concluída
Iteração com gamma 10.000000 e c 0.100000 concluída
Iteração com gamma 10.000000 e c 1.000000 concluída
Iteração com gamma 10.000000 e c 10.000000 concluída
Iteração com gamm

[(0.6897130699787246, 10, 100),
 (0.6972974527054225, 0.1, 10),
 (0.698315220516359, 1, 1),
 (0.7026795468920707, 0.01, 10),
 (0.719872347766086, 0.01, 100)]